In [ ]:
import mmap
import multiprocessing
import time
import os

# Размер буфера общей памяти
BUFFER_SIZE = 1024

# Функция производителя
def producer(shared_buffer):
    # Записываем строку в общую память
    message = "Привет, это сообщение от производителя!"
    print(f"Производитель: отправляю сообщение: {message.decode()}")
    
    # Записываем данные в буфер
    shared_buffer.seek(0)  # Устанавливаем указатель в начало
    shared_buffer.write(message)
    shared_buffer.flush()  # Очищаем буфер
    
    # Ждем немного, чтобы потребитель успел прочитать
    time.sleep(1)

# Функция потребителя
def consumer(shared_buffer):
    # Читаем данные из общей памяти
    shared_buffer.seek(0)  # Устанавливаем указатель в начало
    message = shared_buffer.read(BUFFER_SIZE).strip()
    
    # Выводим полученное сообщение
    print(f"Потребитель: получил сообщение: {message.decode()}")

if __name__ == "__main__":
    # Создаем файл для общей памяти
    with open("shared_memory.bin", "wb+") as f:
        # Создаем буфер общей памяти
        shared_buffer = mmap.mmap(f.fileno(), BUFFER_SIZE, access=mmap.ACCESS_WRITE)
        
        # Создаем процессы
        producer_process = multiprocessing.Process(target=producer, args=(shared_buffer,))
        consumer_process = multiprocessing.Process(target=consumer, args=(shared_buffer,))
        
        try:
            # Запускаем процессы
            producer_process.start()
            consumer_process.start()
            
            # Ждем завершения процессов
            producer_process.join()
            consumer_process.join()
            
        finally:
            # Закрываем общую память
            shared_buffer.close()
            # Удаляем временный файл
            os.remove("shared_memory.bin")